In [ ]:
import sys
from pathlib import Path

parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

from model_finetuning.ModelClassAC_CMR import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
from scipy.io import loadmat
from fairseq_signals.models.wav2vec2.wav2vec2_cmsc import Wav2Vec2CMSCModel
from omegaconf import OmegaConf
import os

In [ ]:
path_to_foundational_model = "path/to/fm" #path where the ECG-FM mimic_iv_ecg_physionet_pretrained.pt model is saved.
path_to_finetuned_weights = "path/to/finetuned" #path where the finetuned model weights are saved
path_to_ecgs = "path/to/ecgs" #path to the ecgs to be evaluated

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}", flush=True)

In [ ]:
ckpt_encoder = torch.load(path_to_foundational_model, weights_only = False)
cfg_encoder = ckpt_encoder['cfg']
cfg_encoder = OmegaConf.create(cfg_encoder["model"])
cfg_encoder["saliency"] = False
encoder = Wav2Vec2CMSCModel(cfg_encoder)

In [ ]:
state_dict = torch.load(path_to_finetuned_weights)
finetuned_model = ECGRegressionModel(encoder, feature_dim=768, num_outputs=4)
finetuned_model.load_state_dict(state_dict)
finetuned_model.to(device)

In [ ]:
ecg_dirs = os.listdir(path_to_ecgs)
test_samples = []
IDs = []
for ecg_file in ecg_dirs:
    ecg_samples = []
    dir = path_to_ecgs + ecg_file
    ecg = loadmat(dir)["feats"]
    ecg_tensor = torch.from_numpy(ecg).float()
    test_samples.append(ecg_tensor)
    IDs.append(ecg_file)

X_test = torch.stack(test_samples, dim=0)
print(X_test.size())  

In [ ]:
test_dataset = TensorDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32)  # Adjust batch_size as needed

# Collect predictions in batches
predictions = []

finetuned_model.eval()
with torch.no_grad():
    for batch in test_loader:
        batch_x = batch[0].to("cuda")
        outputs = finetuned_model(batch_x).cpu().numpy()
        predictions.append(outputs)
    all_predictions = np.concatenate(predictions, axis=0)

In [ ]:
#De-standardize predictions based on mean and standard deviation from train set
means = [38.81489701, 15.40991176, 0.61544391, 0.49352363]
standard_deviations = [11.03477083, 6.94679033, 0.08871907, 0.12588293]

for i in range(4):
    all_predictions[:,i] = all_predictions[:,i] * standard_deviations[i] + means[i]

In [ ]:
results = pd.DataFrame({"ID" : IDs, "Pred. LA max" : all_predictions[:,0], "Pred. LA min" : all_predictions[:,1], "Pred. LAEF" : all_predictions[:,2], "Pred. LALV" : all_predictions[:,3]})